# Running Programs from the Command Line

When you call your programs from the CLI, you can pass arguments to it. In the most basic form this looks like:

```bash
$ python your_program.py arg1 arg2
```

Passing CLI arguments is common praxis in Linux/Unix environments, see the following section for an introduction on how to use the CLI in Linux. In this session, we will learn how to parse CLI arguments and options so that you can run your programs parametrized from the command-line.


## *nix Command Line Primers

  * http://lifehacker.com/5633909/who-needs-a-mouse-learn-to-use-the-command-line-for-almost-anything
  * http://www.makeuseof.com/tag/a-quick-guide-to-get-started-with-the-linux-command-line/
  * http://mvhs-fuhsd.org/java/Units/Unit01/LinuxCommandLinePrimer.pdf


## Parsing CLI Arguments

Arguments are given -separated by spaces- after the name of your program on the CLI. Within your code, you can access them via the `argv` in the `sys` module, where argv `argv[0]` is the script pathname if known and all the following elements of that list are the arguments given to your program.

```python
import sys
from urllib.parse import urlparse
import logging


log_fmt = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(level=logging.DEBUG, format=log_fmt)


def check_args(arguments):
    # This is to be implemented in your programs...
    return True
    
    
def run(arguments):
    if check_args(arguments):
        for idx, argument in enumerate(arguments):
            logging.info('{} argument is {}'.format(idx, argument))
    else:
        print('Usage: python your_script.py arg_1 [arg_2 ...]')
    
    
if __name__ == '__main__':
    # Call me from the CLI for example with:
    # python your_script.py arg_1 [arg_2 ...]
    run(sys.argv[1:])
```


## Parsing CLI Options

Options are given -separated by spaces- after the name of your program and after option names following dashes on the CLI. Within your code, you can parse them using `getopt.getopt` out of  the `argv` in the `sys` module. See the following example:

```python
import sys
import getopt


def usage():
    return 'Usage : cli_opt_demo.py –n <name> or cli_opt_demo.py --name <name>'


def run(arguments):
    try:
        opts, args = getopt.getopt(arguments, "ho:v", ["help", "output="])
    except getopt.GetoptError as err:
        # print help information and exit:
        print(err)  # will print something like "option -a not recognized"
        usage()
        sys.exit(2)

    output = None
    verbose = False
    for option, argument in opts:
        print(option)
        if option == "-v":
            verbose = True
        elif option in ("-h", "--help"):
            print(usage())
            sys.exit()
        elif option in ("-o", "--output"):
            output = argument
        else:
            assert False, "unhandled option"

    print(output)

    
if __name__ == "__main__" :
    run(sys.argv[1:])
```

## Piping Arguments to Your Program

If you want to allow your program to consume data that is piped in from other programs, then you have to let your program read input from `stdin`, which you can access from the module `sys`. Cosider the following example `cli_reverse.py`, which reverses all lines that are piped to it. That is, it is called via for example `cat your.txt | python cli_reverse.py` (for windows users: `type your.txt | python cli_reverse.py`. The program writes it's output again to `stdout` so that it can be piped further on to any other program.

```python
import sys

if __name__ == '__main__':
    input_lines = sys.stdin.read().split('\n')
    output_lines = reversed(input_lines)
    output_str = '\n'.join(output_lines)
    sys.stdout.write(output_str)

```


**OBS:** You can combine CLI arguments and piping support, as the following example illustrates. `cli_replace.py` replaces a match for a regular expression line by line with a replacement text. The regular expression and the replacement text are given as arguments to the program.

```python
# Adapted from: https://wiki.python.org/moin/Powerful%20Python%20One-Liners

import re
import sys


if __name__ == '__main__':
    # Call me from the CLI for example with:
    # printf "uiuiui cat aiaia bumbum\naiaiai" | python cli_replace.py cat dog
    pattern = sys.argv[1]
    substitution = sys.argv[2]

    for line in sys.stdin:
        sys.stdout.write(re.sub(pattern, substitution, line))
```

Running the above program generates the following output:

```bash
$ printf "uiuiui cat aiaia bumbum\naiaiai" | python cli_replace.py cat dog
uiuiui dog aiaia bumbum
aiaiai
```

## Piping stdin and stdout
Piping standard in data to your program and piping standard out data to another program  
Try this out: `cat test.txt | python regexp_find.py "[0-9]" | python line_count.py` to see how lines from test.txt are piped into python running scriptfile regexp_find.py with cli argument "[0-9]" which is writing to standardout which is piped on to another python script: line_count.py

# CLI Arguments the Easy Way

There are quite a few libraries, which allow for parsing CLI arguments and options in a way that you do not have to write repititive code.

My favorite is `(docopt)` http://docopt.org, a CLI description language. After installing it via `conda install docopt` you can let the library create a parser for your CLI arguments out of a module's doc string.

```python
"""Naval Fate.

Usage:
  naval_fate.py ship new <name>...
  naval_fate.py ship <name> move <x> <y> [--speed=<kn>]
  naval_fate.py ship shoot <x> <y>
  naval_fate.py mine (set|remove) <x> <y> [--moored | --drifting]
  naval_fate.py (-h | --help)
  naval_fate.py --version

Options:
  -h --help     Show this screen.
  --version     Show version.
  --speed=<kn>  Speed in knots [default: 10].
  --moored      Moored (anchored) mine.
  --drifting    Drifting mine.

"""
from docopt import docopt


if __name__ == '__main__':
    # run the program for example via
    # python naval_fate.py ship Guardian move 10 50 --speed=20
    arguments = docopt(__doc__, version='Naval Fate 2.0')
    print(arguments)
```

When running the above program you can access the values of your arguments and options from the `arguments` dictionary, which consits the following with respect to the above example.

```python
{'--drifting': False,
 '--help': False,
 '--moored': False,
 '--speed': '20',
 '--version': False,
 '<name>': ['Guardian'],
 '<x>': '10',
 '<y>': '50',
 'mine': False,
 'move': True,
 'new': False,
 'remove': False,
 'set': False,
 'ship': True,
 'shoot': False}
```



# Exercise!!!

  1. Write a program `download_script.py`, which downloads a set of files from the internet. The files to download are given as arguments to your program on the command-line as illustrated in the following:

  ```bash
  $ python download_script.py http://www.gutenberg.org/files/2701/2701-0.txt http://www.gutenberg.org/cache/epub/27525/pg27525.txt
  Downloading file to ./2701-0.txt
  Downloading file to ./pg27525.txt
  ```

    Reuse your `webget` module from exercises in notebook: 02-0c Modules.
  
  2. Modify the above program, so that it can download a list of files from stdin. That is, so that you can reuse the output of one CLI command as input to your program. 

  ```bash
  $ cat list_of_files.txt | python download_script.py
  ```
